In [1]:
import numpy as np

[[0.53535747 0.41675608 0.98922961 0.48258172 0.94529622 0.34727495
  0.18872267 0.88398712 0.35392968 0.44560466]
 [0.49190641 0.14189956 0.7370002  0.66960612 0.6332764  0.6940282
  0.6394224  0.8623481  0.12001771 0.23939769]
 [0.207647   0.01996924 0.25419932 0.92725891 0.61198346 0.11875531
  0.86781856 0.46130609 0.572977   0.28839685]
 [0.6277212  0.70447301 0.74389296 0.22361916 0.48473753 0.18815458
  0.23690646 0.88424848 0.15228795 0.77963565]
 [0.71519777 0.63775781 0.17825317 0.71498745 0.80717772 0.6603122
  0.49112675 0.95177922 0.88739788 0.38601621]]
[[1.53535747 1.41675608 1.98922961 1.48258172 1.94529622 1.34727495
  1.18872267 1.88398712 1.35392968 1.44560466]
 [1.49190641 1.14189956 1.7370002  1.66960612 1.6332764  1.6940282
  1.6394224  1.8623481  1.12001771 1.23939769]
 [1.207647   1.01996924 1.25419932 1.92725891 1.61198346 1.11875531
  1.86781856 1.46130609 1.572977   1.28839685]
 [1.6277212  1.70447301 1.74389296 1.22361916 1.48473753 1.18815458
  1.23690646 1

In [2]:
class SineCosineOptimization():
    def __init__(self):
        ...
        
    def compile(self,obj,dimension,lb,ub,a=2,population_size=100,max_iter=100,optimization="maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.a=a
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    def __initialize_population(self,population_size,dim,lb,ub):
        return lb+np.random.rand(population_size,dim)*(ub-lb)
    
    def __update_R(self,current_iter,max_iter,a):
        r1 = a-a*(current_iter/max_iter)
        r2 = 2*np.pi*np.random.rand()
        r3 = np.random.rand()
        r4 = np.random.rand()
    
        return r1,r2,r3,r4
    
    def __update_population(self,populations,fittest_candidate,r1,r2,r3,r4,lb,ub):
        if r4 < 0.5:
            populations = populations + r1*np.sin(r2)*np.abs(r3*fittest_candidate-populations)
        else:
            populations = populations + r1*np.cos(r2)*np.abs(r3*fittest_candidate-populations)
        
        return np.clip(populations,lb,ub)
    
    def calculate_optimal_value(self):

        #Initialize the variable for  fittest candidate
        fittest_candidate=None
        candidate_evaluations = np.empty([self.population_size])

        #Initialize the random population
        populations = self.__initialize_population(self.population_size,self.dimension,self.lb,self.ub)

        ## This is the iteration loop.
        for i in range(self.max_iter):

            ## This loop will go through the all population of candidate and process them one by one
            for j in range(populations.shape[0]):
                candidate_evaluations[j]=self.objective_function(populations[j,:])


            ## Determining the fittest candidate
            if self.optimization == "maximize":
                fittest_indice = np.argmax(candidate_evaluations)
            elif self.optimization == "minimize":
                fittest_indice = np.argmin(candidate_evaluations)


            ## Updating the fittest candidate

            if fittest_candidate == None:
                fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
            else:
                current_fittest = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
                if self.optimization == "maximize" and fittest_candidate[1] < current_fittest[1]:
                    fittest_candidate = current_fittest

                if self.optimization == "minimize" and fittest_candidate[1] > current_fittest[1]:
                    fittest_candidate = current_fittest

            ## Updating the value of r1,r2,r3, and r4
            r1,r2,r3,r4 = self.__update_R(i,self.max_iter,self.a)

            ## Update the population
            
            for index,value in enumerate(populations):
                populations[index:] = self.__update_population(value,fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)

            populations[index,:] = self.__update_population(populations,fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)

        return fittest_candidate
    
    def one_step_update(self,populations,pop_id,iteration,fittest_candidate=None):
        
        # Initialize candidate evaluation variables
        candidate_evaluations = np.empty([self.population_size])
        
        # Calculate the candidate evaluation using objective function
        for j in range(populations.shape[0]):
            candidate_evaluations[j]=self.objective_function(populations[j,:])
        
         ## Determining the fittest candidate
        if self.optimization == "maximize":
            fittest_indice = np.argmax(candidate_evaluations)
        elif self.optimization == "minimize":
            fittest_indice = np.argmin(candidate_evaluations)
        if fittest_candidate == None:
            fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
        
        r1,r2,r3,r4 = self.__update_R(iteration,self.max_iter,self.a)
        
        populations[pop_id,:] = self.__update_population(populations[pop_id,:],fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)
        
        r
        

### Testing the Algorithm

In [3]:
def obj_f(x):
    return x[0]/x[1]+x[1]+x[2]

In [8]:
lb = np.array([-100,-100,-100])
ub = np.array([100,100,100])
dimension = 3
max_iter=50
population_size=100
model = SineCosineOptimization()
model.compile(obj_f,dimension,lb,ub,max_iter=max_iter,population_size=population_size)
answer = model.calculate_optimal_value()
print(answer)

(array([-4.85413756e-02, -1.54133665e-07,  8.26770574e-02]), 314930.47512924374)


In [9]:
def sums(X):
    return X[0]+X[1]
lb = np.array([-10,-10])
ub = np.array([10,10])
dimension = 2
max_iter=50
population_size=100
model = SineCosineOptimization()
model.compile(sums,dimension,lb,ub,max_iter=max_iter,population_size=population_size)
answer = model.calculate_optimal_value()
print(answer)

(array([10., 10.]), 20.0)
